In [ ]:
pip install tensorflow gradio opencv-python numpy


In [2]:
from google.colab import files
import zipfile, os

# 1) Upload your zip (select your dataset zip file from your PC)
uploaded = files.upload()

# 2) Get uploaded file name automatically
zip_filename = list(uploaded.keys())[0]
print("Uploaded file:", zip_filename)

# 3) Extract it to /content/dataset
extract_path = "/content/dataset"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extracted to:", extract_path)
print("Folders inside /content/dataset:")
print(os.listdir(extract_path))


Saving archive (1).zip to archive (1).zip
Uploaded file: archive (1).zip
Extracted to: /content/dataset
Folders inside /content/dataset:
['test', 'train']


In [3]:
import zipfile, os

zip_filename = list(uploaded.keys())[0]

extract_path = "/content/dataset"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extracted! Folders:", os.listdir(extract_path))


Extracted! Folders: ['test', 'train']


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Base dataset folder
BASE_DIR = "/content/dataset"

print("Folders inside dataset:", os.listdir(BASE_DIR))

# Auto-detect train & test folder names
train_dir = None
test_dir = None

for folder in os.listdir(BASE_DIR):
    lower = folder.lower()
    if "train" in lower:
        train_dir = os.path.join(BASE_DIR, folder)
    if "test" in lower:
        test_dir = os.path.join(BASE_DIR, folder)

print("Detected train folder:", train_dir)
print("Detected test folder:", test_dir)

# Safety check
if train_dir is None or test_dir is None:
    raise Exception("❌ Could not find train or test folder. Please check dataset structure.")

# Image settings
IMG_SIZE = (48, 48)
BATCH_SIZE = 32

# Data generators
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

test_gen = ImageDataGenerator(rescale=1./255)

# Load data
train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_data = test_gen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Class mapping
print("\nClass Indices:", train_data.class_indices)
num_classes = len(train_data.class_indices)


Folders inside dataset: ['test', 'train']
Detected train folder: /content/dataset/train
Detected test folder: /content/dataset/test
Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.

Class Indices: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


In [5]:
from tensorflow.keras import layers, models

IMG_HEIGHT, IMG_WIDTH = 48, 48

# Build CNN model
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Train the model
EPOCHS = 10   # you can increase to 15 for higher accuracy

history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=EPOCHS
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 46, 46, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 21, 21, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 620,487 (2.37 MB)

 Trainable params: 620,039 (2.37 MB)

 Non-trainable params: 448 (1.75 KB)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 202s 220ms/step - accuracy: 0.2467 - loss: 2.0204 - val_accuracy: 0.3685 - val_loss: 1.6187
Epoch 2/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 194s 216ms/step - accuracy: 0.3418 - loss: 1.6445 - val_accuracy: 0.4539 - val_loss: 1.4239
Epoch 3/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 197s 220ms/step - accuracy: 0.3879 - loss: 1.5470 - val_accuracy: 0.4642 - val_loss: 1.3960
Epoch 4/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 196s 219ms/step - accuracy: 0.4182 - loss: 1.4875 - val_accuracy: 0.4866 - val_loss: 1.3278
Epoch 5/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 195s 218ms/step - accuracy: 0.4408 - loss: 1.4528 - val_accuracy: 0.4692 - val_loss: 1.3820
Epoch 6/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 195s 218ms/step - accuracy: 0.4584 - loss: 1.4139 - val_accuracy: 0.5010 - val_loss: 1.3037
Epoch 7/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 191s 213ms/step - accuracy: 0.4710 - loss: 1.3793 - val_accuracy: 0.5132 - val_loss: 1.2714
Epoch 8/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 193s 215ms/step - accuracy: 0.4900 -

In [6]:
model_path = "/content/emotion_model.h5"
model.save(model_path)

print("Model saved at:", model_path)


Model saved at: /content/emotion_model.h5


In [7]:
import gradio as gr
import tensorflow as tf
import numpy as np
import cv2

# Load the saved model
model = tf.keras.models.load_model("/content/emotion_model.h5")

# Get class labels
idx_to_class = {v: k for k, v in train_data.class_indices.items()}
labels = [idx_to_class[i] for i in range(len(idx_to_class))]
print("Emotion Labels:", labels)

IMG_SIZE = (48, 48)

def predict_emotion(img):
    # Resize and normalize
    img_resized = cv2.resize(img, IMG_SIZE)
    img_norm = img_resized / 255.0
    img_input = np.expand_dims(img_norm, axis=0)

    # Predict
    predictions = model.predict(img_input)[0]
    return {labels[i]: float(predictions[i]) for i in range(len(labels))}

# Create Gradio interface
demo = gr.Interface(
    fn=predict_emotion,
    inputs=gr.Image(type="numpy", label="Upload an Image"),
    outputs=gr.Label(num_top_classes=len(labels), label="Predicted Emotion"),
    title="AI Emotion Detection App",
    description="Upload any face image to detect emotion (Happy, Angry, Sad, Neutral, etc.)"
)

demo.launch()


Emotion Labels: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5e2446182d52c99f9f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
